In [3]:
def load_data( training_size = None ):
    images_df = pd.read_csv("Images.csv", sep=";", names=["image_id", "class"])
    edgehistogram_df = pd.read_csv("EdgeHistogram.csv", sep=";", names=["image_id"] + [f"feature{i}" for i in range(9144)])
    df = images_df.join(edgehistogram_df.set_index("image_id"), on="image_id")
    df[df.columns[df.columns != 'class']] = df[df.columns[df.columns != 'class']].apply(pd.to_numeric, downcast='integer', errors='coerce')
    df.fillna(0, inplace=True)
    df = df.iloc[2:]
    df.drop(df[df['class'] == 'BACKGROUND_Google'].index, inplace=True)
    if training_size is not None:
      df = df.groupby("class").head(training_size)
    else :
      pass
    return df

def preprocess_data(df):
    encoder = OneHotEncoder(handle_unknown='ignore', sparse='auto')
    class_one_hot = encoder.fit_transform(df[['class']]).toarray()
    class_one_hot = pd.DataFrame(class_one_hot)
    class_one_hot.columns = encoder.get_feature_names(['class_'])
    X = df.drop(columns=["image_id", "class"])
    y = df["class"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def train_and_predict(X_train, X_test, y_train, y_test):
    param_grid = { 'C': [3], 'gamma': [0.017], 'kernel': ['rbf'] }
    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3 )
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    y_pred = grid.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100}%")
    #conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    # Plot the confusion matrix
    #cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])
    #cm_display.plot()
    #plt.show()
    # Get the predicted probability scores for the positive class
 
if __name__ == "__main__":
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import GridSearchCV
    from sklearn import metrics
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_auc_score, roc_curve
    
    # Pass the training size inside load_data() function  
    df = load_data()
    X_train, X_test, y_train, y_test = preprocess_data(df)
    train_and_predict(X_train, X_test, y_train, y_test)

C:\Everything\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ......C=3, gamma=0.017, kernel=rbf;, score=0.601 total time= 5.5min
[CV 2/5] END ......C=3, gamma=0.017, kernel=rbf;, score=0.615 total time= 5.5min
[CV 3/5] END ......C=3, gamma=0.017, kernel=rbf;, score=0.616 total time= 5.5min
[CV 4/5] END ......C=3, gamma=0.017, kernel=rbf;, score=0.611 total time= 6.1min
[CV 5/5] END ......C=3, gamma=0.017, kernel=rbf;, score=0.624 total time= 6.4min
{'C': 3, 'gamma': 0.017, 'kernel': 'rbf'}
0.6133162612035852
Accuracy: 61.52073732718893%
